# Arquivo com as funções bases para a modelagem

In [ ]:
#função para ler os arquivos variant disponibilizados e convertendo em dataframe
def convert_variant_df(read):
    lista = open(read, "r",encoding="utf8").readlines()
    #Esta lista possui \n junto ao texto, entao vamos remover
    lista_nova = [texto.split(sep="\n")[0].split(",") for texto in lista]
    df = pd.DataFrame(lista_nova[1:],columns=lista_nova[0])
    return(df)

In [ ]:
#função para converter os demais arquivos em dataframe
def convert__df(read):
    #separa o texto pelo delimitador ||
    lista = re.split('([0-9]+)(\|\|)',open(read, "r",encoding="utf8").read())
    #Remove da lista os elementos ||
    lista = [elemento for elemento in lista if elemento != "||"]
    #Detecta o titulo do df
    titulo = lista[0].split("\n")[0].split(",")
    lista_nova= [[lista[index+1],lista[index+2]] for index in range(0,len(lista[1:])-1,2)]
    
    df = pd.DataFrame(lista_nova,columns=titulo)
    return(df) 

In [ ]:
# Criando uma função que retorna um dataframe de descrição de dados (tal qual a função describe do pacote explore do R)
def explore_describe(df):
    df_out = pd.DataFrame(columns = ['variable','type','na' ,'na_pct' ,'unique','min', 'quat25','median','mean', \
                                     'quat75','max','std','skewness','kurtosis','media_desvio'])
    df_out['variable'] = df.columns
    df_out['type'] = df.dtypes.values
    df_out['na'] = [sum(df[coluna].isna()) for coluna in df.columns]
    df_out['na_pct'] = [str(round(100*sum(df[coluna].isna())/df.shape[0],1))+'%' for coluna in df.columns]
    df_out['unique'] = [len(df[coluna].unique()) for coluna in df.columns]
    df_out['min']  = [round(min(df[coluna]),2) if 'int' in str(df[coluna].dtype) or 'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['mean'] = [round(df[coluna].mean(),2) if 'int' in str(df[coluna].dtype) or \
                      'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['max']  = [round(max(df[coluna]),2) if 'int' in str(df[coluna].dtype) or 'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['std'] = [round(df[coluna].std(),2) if 'int' in str(df[coluna].dtype) or \
                      'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['quat25'] = [round(df[coluna].quantile(0.25),2) if 'int' in str(df[coluna].dtype) or \
                      'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['quat75'] = [round(df[coluna].quantile(0.75),2) if 'int' in str(df[coluna].dtype) or \
                      'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['median'] = [round(df[coluna].quantile(0.5),2) if 'int' in str(df[coluna].dtype) or \
                      'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['skewness'] = [round(df[coluna].skew(),2) if 'int' in str(df[coluna].dtype) or \
                          'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['kurtosis'] = [round(df[coluna].kurt(),2) if 'int' in str(df[coluna].dtype) or \
                          'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    
    df_out_media_desvio_list = []
    for coluna in df.columns:
        if(('int' in str(df[coluna].dtype)) or ('float' in str(df[coluna].dtype)) ):
            if((all(df[coluna] == 0)) or (df[coluna].std() == 0)):
                df_out_media_desvio_list.append(0)
            else:
                df_out_media_desvio_list.append(round(df[coluna].mean()/df[coluna].std(),2))
        else:
            df_out_media_desvio_list.append('-')
    
    df_out['media_desvio'] = df_out_media_desvio_list
    return(df_out)

In [ ]:
#função para remover caracteres não ASCII
def removeNoAscii(s):
    return "".join(i for i in s if ord(i) < 128)

In [ ]:
#Função para gerar corpus (lista de documentos)
def corpusnization(text):
    #Removendo a pontuação e tokenizando
    nopunct_token = nltk.tokenize.regexp_tokenize(text.lower(),"[\w']+")
   
    #Removendo stopwords
    token_no_stopwords = [word for word in nopunct_token if word not in stopwords.words('english')]
    
    #Stemming
    #cooking -> cook
    token_stem = [PorterStemmer().stem(token) for token in token_no_stopwords]
    
    #Lemmatization
    #mice -> mouse
    token_final = [WordNetLemmatizer().lemmatize(token) for token in token_stem]
    return(token_stem)

In [ ]:
#função para criar o set de palavras (em ordem alfabética e sem repetição)
def cria_listagem_palavras(corpus):
    listagem = set()
    for i in corpus:
        listagem = listagem.union(set(i))
    return(listagem)

In [ ]:
#Esta função irá retornar uma estrutra para o dicionario de palavras, atribuindo indices a elas
def cria_dicionario_palavras(listagem):
    dicionario = {i:j for j,i in enumerate(listagem)}
    return(dicionario)

In [ ]:
# Vamos contar a quantidade de palavras em cada elemento do corpus
def conta_palavras_corpus(elem_corpus):
    return dict(Counter(elem_corpus))

In [ ]:
#Função para criar o dicionario de palavras
def df_dict(listagem,bag_words_corpus):
    return {token: sum([token in doc.keys() for doc in bag_words_corpus]) for token in listagem}
    #total_palavras = [{i:0 for i in listagem}]
    #total_palavras.extend(bag_words_corpus)
    #return(dict(reduce(lambda x,y: Counter(x) + Counter(y), total_palavras)))
    

In [ ]:
#tf(termo,documento) = contagem de termo em documento / número de palavras em documento
#idf (termo) = log (N / (df + 1))

#tf-idf (termo, documento) = tf (termo, documento) * log (N / (df + 1))
def calcula_tf_e_idf(bow, df,N):
    #Calculando a frequencia do termo para cada documento
    tf = [{key:t/sum(documento.values()) for key,t in documento.items()} for documento in bow]
    idf = {chave: np.log(N/(valor+1))  for chave,valor in df.items()}
    return (tf,idf)
    

In [ ]:
def tf_idf(lista_tf,dict_idf):
    return [{chave: tf*dict_idf[chave] for chave,tf in doc.items()} for doc in lista_tf]

In [ ]:
# Função para criar nuvem de palavras
def word_cloud_plot(classe,df,bag_words):
    indices = df[df["Class"] == classe].index
    bag_words_filtrada = [bag_words[i] for i in indices]
    dicionario_classe = reduce(lambda x,y: Counter(x) + Counter(y), bag_words_filtrada)
    wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                min_font_size = 10).generate_from_frequencies(dicionario_classe)
    return(wordcloud)
    #plt.imshow(wordcloud)
    #plt.axis("off")
    #plt.show()
    #return({k: v for k, v in sorted(dicionario_classe.items(), key=lambda item: item[1],reverse=True)})

In [ ]:
#Função que cria uma nuvem de palvras baseado não na frequencia, mas sim, no TF IDF
def word_cloud_plot_tfidf(classe,df,lista_tfidf):
    indices = df[df["Class"] == classe].index
    lista_tfidf_filtrada = [lista_tfidf[i] for i in indices]
    dicionario_classe_tf_idf = reduce(lambda x, y: dict((chave, valor + y[chave]) if chave in y.keys() else (chave, valor) for chave, valor in x.items()), tfidf)
    wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                min_font_size = 10).generate_from_frequencies(dicionario_classe_tf_idf)
    return(wordcloud)

In [ ]:
#Função para plotar a nuvem e palavras
def plota_wordcloud(func,df,lista_dicionario_palavras):
    fig = plt.figure(figsize=(20,10))
    for i in range(1,10):
        ax = fig.add_subplot(3,3,i)
        ax.imshow(func(i,df_training,bag_words_corpus))
        ax.set_title(str(i))
        plt.axis("off")
    plt.show()

In [ ]:
# Para criar o modelo base, precisamos de uma base de dados inicial. Para isso, vamos criar o dataset usando as palavras da listagem
def matriz_esparca(dicionario,tfidf_,N):
    S = sp.dok_matrix((N,len(dicionario)),dtype = np.float32)
    for i,doc in enumerate(tfidf_):
        for chave,valor in doc.items():
            S[i,dicionario[chave]] = valor
    
    return S

In [ ]:
#Função para converter a matriz esparça em tfSparce
def convert_matriz_esparca_tfSparce(M):
    coo = M.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return(tf.SparseTensor(indices, coo.data, coo.shape))